In [79]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import re
import pandas as pd
import seaborn as sns
import datetime
from tqdm import tqdm
import spacy
import sklearn
from termcolor import colored

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

In [4]:
f = open('./data/sentiment/positive')
pos = f.read()
f.close()

f = open('./data/sentiment/negative')
neg = f.read()
f.close()

In [16]:
# Load Spacy word embeddings
word_embeddings = spacy.load('en', vectors='glove.6B.300d.txt')

In [46]:
# Create a function to get vector format data for a sequence
def sequence_to_data(seq):
    seq = unicode(seq)
    data = np.array([word_embeddings(ix).vector for ix in seq.split()])
    return np.expand_dims(data, 0)
    
q = sequence_to_data(u'hello! what is the date today?')

In [151]:
df = pd.DataFrame([], columns=['text', 'score'])
for ix in pos.split('\n'):
    text = ix.strip().lower()
    if len(text) > 1:
        df = df.append({'text': text, 'score': 1}, ignore_index=True)
    # print sequence_to_data(ix.strip().lower()).shape

for ix in neg.split('\n'):
    text = ix.strip().lower()
    if len(text) > 1:
        df = df.append({'text': text, 'score': 0}, ignore_index=True)

In [152]:
df = sklearn.utils.shuffle(df).reset_index(drop=True)

In [153]:
df.head()

,text,score
0,as the dominant christine sylvie testud is ic...,1.0
1,even with its 50million us budget pinocchio n...,0.0
2,in theory a middleaged romance pairing claybu...,0.0
3,excellent performances from jacqueline bisset ...,1.0
4,the enormous comic potential of an oafish idio...,0.0


In [154]:
# df.head(10)
# Printing colored text (Useful later)
print colored("hello red world", 'blue')# print 'a'

hello red world


In [166]:
class SeqModel(nn.Module):
    def __init__(self, in_shape=None, out_shape=None, hidden_shape=None):
        super(SeqModel, self).__init__()
        self.in_shape = in_shape
        self.out_shape = out_shape
        self.hidden_shape = hidden_shape
        self.n_layers = 1
        
        self.rnn = nn.LSTM(
            input_size=self.in_shape,
            hidden_size=self.hidden_shape,
            num_layers=self.n_layers,
            batch_first=True
        )
        self.out = nn.Linear(self.hidden_shape, self.out_shape)
    
    def forward(self, x, h):
        r_out, h_state = self.rnn(x, h)
        last_out = r_out[:, -1, :]
        y = F.softmax(self.out(last_out))
        return y
    
    def predict(self, x):
        h_state = self.init_hidden(1, gpu=False)
        
        x = sequence_to_data(x)
        pred = self.forward(torch.FloatTensor(x), h_state)
        
        return pred
            
    def init_hidden(self, batch_size, gpu=True):
        if gpu:
            return (Variable(torch.zeros(self.n_layers, batch_size, self.hidden_shape).cuda()),
                    Variable(torch.zeros(self.n_layers, batch_size, self.hidden_shape)).cuda())
        return (Variable(torch.zeros(self.n_layers, batch_size, self.hidden_shape)),
                Variable(torch.zeros(self.n_layers, batch_size, self.hidden_shape)))

In [172]:
model = SeqModel(in_shape=300, hidden_shape=256, out_shape=2)
print model

SeqModel(
  (rnn): LSTM(300, 256, batch_first=True)
  (out): Linear(in_features=256, out_features=2, bias=True)
)


In [174]:
model.predict('hello world')

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor([[ 0.5098,  0.4902]])

In [157]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [180]:
# Set to train mode
# model.cuda()
model.train()
N = min(200, df.shape[0])

for epoch in range(10):
    total_loss = 0
    for ix in range(N):        
        h_state = model.init_hidden(1, gpu=False)
        seq = df.text[ix]
        target = np.array([df.score[ix]], dtype=np.int)
        
        data_seq = sequence_to_data(seq)
        
        try:
            x = Variable(torch.FloatTensor(data_seq), requires_grad=True)
            y = Variable(torch.LongTensor(target))
        except:
            print seq
            print data_seq.shape
            x = Variable(torch.FloatTensor(data_seq), requires_grad=True)
            y = Variable(torch.LongTensor(target))
        
        pred = model(x, h_state)
        loss = criterion(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss
        if ix%(N/5) == 0:
            print 'Loss: {} at Epoch: {} | Seq: {}'.format(loss, epoch, ix)
        
    print "Overall Average Loss: {} at Epoch: {}".format(total_loss / float(N), epoch)
    
    # Save model checkpoints
    if epoch % 10 == 0:
        torch.save(model.state_dict(), "./data/checkpoints/seq_lstm/model_256h_epoch_{}.ckpt".format(epoch))

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Loss: 0.694595098495 at Epoch: 0 | Seq: 0
Loss: 0.705835878849 at Epoch: 0 | Seq: 40
Loss: 0.68001717329 at Epoch: 0 | Seq: 80
Loss: 0.704322457314 at Epoch: 0 | Seq: 120
Loss: 0.703839838505 at Epoch: 0 | Seq: 160
Overall Average Loss: 0.692897617817 at Epoch: 0
Loss: 0.694595098495 at Epoch: 1 | Seq: 0
Loss: 0.705835878849 at Epoch: 1 | Seq: 40
Loss: 0.68001717329 at Epoch: 1 | Seq: 80
Loss: 0.704322457314 at Epoch: 1 | Seq: 120
Loss: 0.703839838505 at Epoch: 1 | Seq: 160
Overall Average Loss: 0.692897617817 at Epoch: 1
Loss: 0.694595098495 at Epoch: 2 | Seq: 0
Loss: 0.705835878849 at Epoch: 2 | Seq: 40
Loss: 0.68001717329 at Epoch: 2 | Seq: 80
Loss: 0.704322457314 at Epoch: 2 | Seq: 120
Loss: 0.703839838505 at Epoch: 2 | Seq: 160
Overall Average Loss: 0.692897617817 at Epoch: 2
Loss: 0.694595098495 at Epoch: 3 | Seq: 0


KeyboardInterrupt: 